# Chapter 5. Statisticcal Tests in Julia

🚧 

In [1]:
using Pkg; Pkg.add("HypothesisTests")
using HypothesisTests, CSV, DataFrames, RDatasets

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [2]:
df = dataset("datasets", "iris")
first(df, 5)

Row,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
,Float64,Float64,Float64,Float64,Cat…
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa


## T-tests

### One-sample

In [14]:
mu = 20
OneSampleTTest(df.PetalLength, mu)

One sample t-test
-----------------
Population details:
    parameter of interest:   Mean
    value under h_0:         20
    point estimate:          3.758
    95% confidence interval: (3.473, 4.043)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           <1e-99

Details:
    number of observations:   150
    t-statistic:              -112.68524392285155
    degrees of freedom:       149
    empirical standard error: 0.144135997177411


### Two-samples

In [13]:
UnequalVarianceTTest(df.PetalLength, df.PetalWidth)

Two sample t-test (unequal variance)
------------------------------------
Population details:
    parameter of interest:   Mean difference
    value under h_0:         0
    point estimate:          2.55867
    95% confidence interval: (2.249, 2.868)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           <1e-37

Details:
    number of observations:   [150,150]
    t-statistic:              16.29738511857255
    degrees of freedom:       202.6934059810404
    empirical standard error: 0.1569986011897579


### Paired-samples

In [11]:
EqualVarianceTTest(df.PetalLength, df.PetalWidth)

Two sample t-test (equal variance)
----------------------------------
Population details:
    parameter of interest:   Mean difference
    value under h_0:         0
    point estimate:          2.55867
    95% confidence interval: (2.25, 2.868)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           <1e-42

Details:
    number of observations:   [150,150]
    t-statistic:              16.297385118572546
    degrees of freedom:       298
    empirical standard error: 0.15699860118975792


Attention: the usage of a paired sample test is not valid here, just for illustating how to use the function.

## ANOVA

In [30]:
g1 = df[df.Species .== "setosa", :PetalLength]
g2 = df[df.Species .!= "setosa", :PetalLength]

groups = [g1, g2]
OneWayANOVATest(groups...)

One-way analysis of variance (ANOVA) test
-----------------------------------------
Population details:
    parameter of interest:   Means
    value under h_0:         "all equal"
    point estimate:          NaN

Test summary:
    outcome with 95% confidence: reject h_0
    p-value:                     <1e-65

Details:
    number of observations: [50, 100]
    F statistic:            954.682
    degrees of freedom:     (1, 148)


# Models in Julia

In [31]:
Pkg.add("GLM"); using GLM

   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [38e38edf] + GLM v1.9.0
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


### 1. Multivariate Linear Regression

In [33]:
formula = @formula(PetalLength ~ PetalWidth)
lm(formula, df)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

PetalLength ~ 1 + PetalWidth

Coefficients:
───────────────────────────────────────────────────────────────────────
               Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────
(Intercept)  1.08356   0.072967   14.85    <1e-30   0.939366    1.22775
PetalWidth   2.22994   0.0513962  43.39    <1e-85   2.12838     2.33151
───────────────────────────────────────────────────────────────────────

### 2. Fixed-effects Models (fast)

In [34]:
Pkg.add("FixedEffectModels"); using FixedEffectModels

   Resolving package versions...
   Installed FixedEffects ────── v2.4.0
   Installed GroupedArrays ───── v0.3.3
   Installed Vcov ────────────── v0.8.1
   Installed FixedEffectModels ─ v1.11.0
    Updating `~/.julia/environments/v1.11/Project.toml`
  [9d5cd8c9] + FixedEffectModels v1.11.0
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [9d5cd8c9] + FixedEffectModels v1.11.0
  [c8885935] + FixedEffects v2.4.0
  [6407cd72] + GroupedArrays v0.3.3
  [ec2bfdc2] + Vcov v0.8.1
Precompiling project...
   1423.7 ms  ✓ GroupedArrays
   2298.4 ms  ✓ FixedEffects
   8497.7 ms  ✓ Vcov
  12168.1 ms  ✓ FixedEffectModels
  4 dependencies successfully precompiled in 23 seconds. 511 already precompiled.


In [36]:
formula = @formula(PetalLength ~ PetalWidth + fe(Species))
reg(df, formula, Vcov.cluster(:Species))

                            FixedEffectModel                            
Number of obs:                  150  Converged:                     true
dof (model):                      1  dof (residuals):                  3
R²:                           0.955  R² adjusted:                  0.954
F-statistic:                5.65297  P-value:                      0.098
R² within:                    0.235  Iterations:                       1
            Estimate  Std. Error  t-stat  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────
PetalWidth   1.01871    0.428463  2.3776    0.0978  -0.344848    2.38227
